In [ ]:
import pymc as pm
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
import numpy as np
import xarray as xr
from ridgeplot import ridgeplot
import plotly.express as px
import warnings

In [ ]:
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)
az.style.use("arviz-darkgrid")
warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
df = pd.read_csv("./data/writing_center_v2.csv")
df.rename(columns={'Ethnicity_Hispanic / Latino':'Ethnicity_Hispanic', 'Ethnicity_Mixed Ethnicity':'Ethnicity_Mixed','Ethnicity_White, Non-Hispanic':'Ethnicity_White'},inplace=True)
df['Main_Course_SuccessFlag'] = df['Main_Course_SuccessFlag'].astype(int).values

## Plot Functions

In [ ]:
def plot_parameters(trace, features, trace_plot=True, forest_plot=True):
  '''
  Can be used for complete-pooling, no-pooling and multilevel.
  '''
  features_to_include = ['beta_' + feature for feature in features]
  if trace_plot:
    az.plot_trace(trace, var_names=["beta0"] + features_to_include)
  if forest_plot:
    az.plot_forest(trace, var_names=["beta0"] + features_to_include, combined=True)

def plot_latent_parameters(trace, features, trace_plot=True, forest_plot=True):
  '''
  Can only be used for multilevel.
  '''
  features_to_include_mu = ['beta_' + feature  + '_mu' for feature in features]
  features_to_include_sigma = ['beta_' + feature + '_sigma' for feature in features]
  if trace_plot:
    az.plot_trace(trace, var_names=["mu_beta0", "sigma_beta0"] + features_to_include_mu + features_to_include_sigma)
  if forest_plot:
    az.plot_forest(trace, var_names=["mu_beta0", "sigma_beta0"] + features_to_include_mu + features_to_include_sigma, combined=True)

def plot_structure(model):
  '''
  Can be used for complete-pooling, no-pooling and multilevel.
  '''
  pm.model_to_graphviz(model)

# Complete Pooling Functions

In [ ]:
def complete_pooling(features, df, save_name:str, rng, cv=False):
  coords = {"obs_id": df.index.values}

  with pm.Model(coords=coords) as binomial_regression_model:
      # Define the design matrix (X)
      X = pm.Data("X", df[features].values,
                  dims=["obs_id", "predictor"])

      # Priors
      beta0 = pm.Normal("beta0", mu=0, sigma=10)

      beta_distributions = {}
      # Create Normal distributions dynamically
      for feature in features:
          beta_name = 'beta_' + feature
          beta_distributions[beta_name] = pm.Normal(str(beta_name), mu=0, sigma=10)

      # Linear model with beta0 + linear combination of features-coefficients and feature-data
      mu = beta0
      for feature in features:
        mu += beta_distributions['beta_' + feature] * X[:, features.index(feature)]

      p = pm.Deterministic("p", pm.math.invlogit(mu), dims="obs_id")

      # Likelihood
      pm.Binomial("y", n=1, p=p, observed=df["Main_Course_SuccessFlag"], dims="obs_id")


  # Inference
  with binomial_regression_model:
      trace = pm.sample(2000, tune=1000,return_inferencedata=True, random_seed=rng)
      pm.compute_log_likelihood(trace) # used for model comparison
  
      if cv == False:  
        #save trace
        trace.to_netcdf("./traces/"+save_name+"_complete_pooling.nc")

  return trace, binomial_regression_model

## No Pooling Functions

In [ ]:
def no_pooling(features, df, save_name:str, rng, cv=False):
    instructor_idxs, instructors = pd.factorize(df.Instructor_ID)
    num_instructors = len(instructors)

    coords = {
        "instructor": instructors,
        "obs_id": np.arange(len(instructor_idxs)),
        "predictor": features
    }

    with pm.Model(coords=coords) as binomial_regression_model:
        # Define instructor
        instructor_idx = pm.Data("instructor_idx", instructor_idxs, dims="obs_id")

        # Define the design matrix (X) with instructor-specific predictors
        X = pm.Data("X", df[features].values, dims=["obs_id", "predictor"])

        # Priors
        beta0 = pm.Normal("beta0", 0, sigma=10, dims="instructor")  # intercept varying by instructor

        beta_distributions = {}
        # Create Normal distributions dynamically
        for feature in features:
            beta_name = 'beta_' + feature
            beta_distributions[beta_name] = pm.Normal(str(beta_name), 0, sigma=10, dims="instructor")  # all betas varying by instructor

        # Linear model
        mu = beta0[instructor_idx]
        for feature in features:
            mu += beta_distributions['beta_' + feature][instructor_idx] * X[:, features.index(feature)]

        p = pm.Deterministic("p", pm.math.invlogit(mu), dims="obs_id")

        # Likelihood
        pm.Binomial("y", n=1, p=p, observed=df["Main_Course_SuccessFlag"], dims="obs_id")

    # Inference
    with binomial_regression_model:
        trace = pm.sample(2000, tune=1000, return_inferencedata=True, random_seed=rng)
        pm.compute_log_likelihood(trace)  # used for model comparison
    
        if cv == False:
            #save trace
            trace.to_netcdf("./traces/"+save_name+"_no_pooling.nc")
    return trace, binomial_regression_model

## Multilevel Functions (Hierarchical Models)

In [ ]:
def multilevel(features, df, save_name:str, rng, cv=False):
    instructor_idxs, instructors = pd.factorize(df.Instructor_ID)
    num_instructors = len(instructors)

    coords = {
        "instructor": instructors,
        "obs_id": np.arange(len(instructor_idxs)),
        "predictor": features
    }

    with pm.Model(coords=coords) as binomial_regression_model:
        # Define instructor
        instructor_idx = pm.Data("instructor_idx", instructor_idxs, dims="obs_id")

        # Define the design matrix (X) with instructor-specific predictors
        X = pm.Data("X", df[features].values, dims=["obs_id", "predictor"])

        # Hyperpriors for instructor intercept
        mu_beta0 = pm.Normal("mu_beta0", mu=0.0, sigma=1)
        sigma_beta0 = pm.HalfNormal("sigma_beta0", 1)
        # Hyperpriors for intructor betas
        hyperpriors_mu = {}
        hyperpriors_sigma =  {}
        for feature in features:
            beta_name = 'beta_' + feature
            hyperpriors_mu[beta_name] = pm.Normal(str(beta_name)+'_mu', mu=0, sigma=1)
            hyperpriors_sigma[beta_name] = pm.HalfNormal(str(beta_name)+'_sigma', 1)

        # Priors
        beta0 = pm.Normal("beta0", mu_beta0, sigma_beta0, dims="instructor")  # mu and sigma follows from multilevel and intercept varying by instructor
        beta_distributions = {}
        # Create Normal distributions dynamically
        for feature in features:
            beta_name = 'beta_' + feature
            beta_distributions[beta_name] = pm.Normal(str(beta_name),
                                                      mu = hyperpriors_mu[beta_name], #mu follows from multilevel
                                                      sigma=hyperpriors_sigma[beta_name], # sigma follows from multilevel
                                                      dims="instructor")  # all betas varying by instructor
        # Linear model
        mu = beta0[instructor_idx]
        for feature in features:
            mu += beta_distributions['beta_' + feature][instructor_idx] * X[:, features.index(feature)]

        p = pm.Deterministic("p", pm.math.invlogit(mu), dims="obs_id")

        # Likelihood
        pm.Binomial("y", n=1, p=p, observed=df["Main_Course_SuccessFlag"], dims="obs_id")

    # Inference
    with binomial_regression_model:
        trace = pm.sample(6000, tune=2000, return_inferencedata=True, random_seed=rng)
        pm.compute_log_likelihood(trace)  # used for model comparison
    
        if cv == False:
            #save trace
            trace.to_netcdf("./traces/"+save_name+"_multilevel.nc")

    return trace, binomial_regression_model

## 2c - 5 fold CV Function

In [ ]:
# Log loss
# https://medium.com/analytics-vidhya/binary-crossentropy-in-its-core-35bcecf27a8a
def binary_cross_entropy_loss(true, predicted):
  binary_cross_entropy = -np.mean(true * np.log(predicted) + (1 - true) * np.log(1 - predicted))
  return binary_cross_entropy

In [ ]:
def cv_error(features, df, complete_pooling_=True, no_pooling_=True, multilevel_=True):
  # Create StratifiedKFold object with 5 folds
  target_instructor_stratify = df["Main_Course_SuccessFlag"].astype(str) + "_" + df["Instructor_ID"].astype(str)
  stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

  losses_complete_pooling = []
  losses_no_pooling = []
  losses_multilevel = []

  # Iterate over the folds
  for fold, (train_index, test_index) in enumerate(stratified_kfold.split(df[features], target_instructor_stratify)):
        train_data = df.iloc[train_index]
        test_data = df.iloc[test_index]
        test_features = test_data[features+['Instructor_ID']]
        test_target = test_data[['Main_Course_SuccessFlag', 'Instructor_ID']]


        ######################### Complete Pooling ###########################
        if complete_pooling_:
          trace_train, model_train = complete_pooling(features, train_data, 'fill', cv=True)
          #grab posterior mean betas for the features - takes quite some time
          posterior_mean_beta0 = pm.summary(trace_train)['mean']['beta0']
          posterior_mean_betas = {}
          for feature in features:
            beta_name = 'beta_' + feature
            posterior_mean_betas[beta_name] = pm.summary(trace_train)['mean'][beta_name]
          # make the predictions on the test_features
          log_odds_predictions = posterior_mean_beta0
          for feature in features:
            log_odds_predictions += posterior_mean_betas['beta_' + feature] * test_features[feature] # dot product of betas and test_features
          prob_predictions = 1 / (1 + np.exp(-log_odds_predictions))
          loss = binary_cross_entropy_loss(test_target.loc[:,'Main_Course_SuccessFlag'], prob_predictions)
          losses_complete_pooling.append(loss)

        #################### No-pooling #######################
        if no_pooling_:
          trace_train, model_train = no_pooling(features, train_data, 'fill', cv=True)
          for instructor in df['Instructor_ID'].unique().tolist():
            posterior_mean_beta0 = pm.summary(trace_train)['mean']['beta0['+str(instructor)+']']
            posterior_mean_betas = {}
            for feature in features:
              beta_name = 'beta_' + feature + '[' + str(instructor) + ']'
              posterior_mean_betas[beta_name] = pm.summary(trace_train)['mean'][beta_name]
            # make the predictions on the test_features
            log_odds_predictions = posterior_mean_beta0
            for feature in features:
              log_odds_predictions += posterior_mean_betas['beta_' + feature + '[' + str(instructor) + ']'] * test_features[test_features['Instructor_ID']==instructor][feature] # dot product of betas and test_features for instructor
            prob_predictions = 1 / (1 + np.exp(-log_odds_predictions))
            loss = binary_cross_entropy_loss(test_target[test_target['Instructor_ID']==instructor]['Main_Course_SuccessFlag'], prob_predictions) # loss for instructor
            losses_no_pooling.append(loss)

        #################### Multilevel #######################
        if multilevel_:
          trace_train, model_train = multilevel(features, train_data, 'fill', cv=True)
          for instructor in df['Instructor_ID'].unique().tolist():
            posterior_mean_beta0 = pm.summary(trace_train)['mean']['beta0['+str(instructor)+']']
            posterior_mean_betas = {}
            for feature in features:
              beta_name = 'beta_' + feature + '[' + str(instructor) + ']'
              posterior_mean_betas[beta_name] = pm.summary(trace_train)['mean'][beta_name]
            # make the predictions on the test_features
            log_odds_predictions = posterior_mean_beta0
            for feature in features:
              log_odds_predictions += posterior_mean_betas['beta_' + feature + '[' + str(instructor) + ']'] * test_features[test_features['Instructor_ID']==instructor][feature] # dot product of betas and test_features for instructor
            prob_predictions = 1 / (1 + np.exp(-log_odds_predictions))
            loss = binary_cross_entropy_loss(test_target[test_target['Instructor_ID']==instructor]['Main_Course_SuccessFlag'], prob_predictions) # loss for instructor
            losses_multilevel.append(loss)

  return losses_complete_pooling, losses_no_pooling, losses_multilevel


# FITTING MODELS

## Fitting Gender Model

In [ ]:
gender_features = ['Gender_Male', 'Gender_Female']
# gender_complete_pooling_trace, gender_complete_pooling_model = complete_pooling(gender_features, df, 'gender',rng)
# gender_no_pooling_trace, gender_no_pooling_model = no_pooling(gender_features, df, 'gender', rng)
gender_multilevel_trace, gender_multilevel_model = multilevel(gender_features, df, 'gender_constrained', rng)

In [ ]:
# Load the models

# gender_complete_pooling_trace = az.from_netcdf("./traces/gender_complete_pooling.nc")
# gender_no_pooling_trace = az.from_netcdf("./traces/gender_no_pooling.nc")
# gender_multilevel_trace = az.from_netcdf("./traces/gender_multilevel.nc")

# ethnicity_multilevel_trace = az.from_netcdf("./traces/ethnicity_multilevel.nc")


In [ ]:
plot_latent_parameters(gender_multilevel_trace, gender_features)

In [ ]:
plot_parameters(gender_no_pooling_trace, gender_features)

## Fitting Ethnicity Model

In [ ]:
ethnicity_features = ['Ethnicity_White', 'Ethnicity_Asian', 'Ethnicity_Hispanic', 'Ethnicity_Mixed']
ethnicity_complete_pooling_trace, ethnicity_complete_pooling_model = complete_pooling(ethnicity_features, df, 'ethnicity', rng)
ethnicity_no_pooling_trace, ethnicity_no_pooling_model = no_pooling(ethnicity_features, df, 'ethnicity', rng)
ethnicity_multilevel_trace, ethnicity_multilevel_model = multilevel(ethnicity_features, df, 'ethnicity', rng)

In [ ]:
plot_parameters(ethnicity_no_pooling_trace, ethnicity_features)

In [ ]:
plot_latent_parameters(ethnicity_multilevel_trace, ethnicity_features)

# Fitting Adjusted Model

In [ ]:
df.describe()

In [ ]:
adjusted_features = ['Gender_Male', 'Gender_Female', 'Ethnicity_White', 'Ethnicity_Asian', 'Ethnicity_Hispanic', 'Ethnicity_Mixed',
                      'Age', 'FirstGen', 'Military', 'FosterYouth', 'DSPS','FinAid', 'Units_Attempted_Beg_Of_Term', 
                      'TermUnitsAttempted', 'K12_Student', 'First_Time_College_Student', 'Nonresident_Tuition_Exempt', 'International', 'Nonresident', 
                      'WR_Center', 'Online', 'N_Center_Visits', 'Center_Attendance_Hours', 'N_Conf', 'WR_Center_FailFlag']
# adjusted_complete_pooling_trace, adjusted_complete_pooling_model = complete_pooling(adjusted_features, df, 'adjusted', rng)
adjusted_no_pooling_trace, adjusted_no_pooling_model = no_pooling(adjusted_features, df.iloc[0:50], 'adjusted', rng)
# adjusted_multilevel_trace, adjusted_multilevel_model = multilevel(adjusted_features, df, 'adjusted', rng)

In [ ]:
plot_parameters(adjusted_no_pooling_trace, adjusted_features)

In [ ]:
plot_latent_parameters(adjusted_multilevel_trace, adjusted_features)

# EXPERIMENTS

## 2a & 2b Loo and WAIC from Pymc3

In [ ]:
az.compare({'gender_complete_pooling': gender_complete_pooling_trace,
            'gender_no_pooling': gender_no_pooling_trace,
            'gender_multilevel': gender_multilevel_trace
            })

In [ ]:
az.plot_compare(az.compare({'gender_complete_pooling': gender_complete_pooling_trace,
            'gender_no_pooling': gender_no_pooling_trace,
            'gender_multilevel': gender_multilevel_trace
            }), insample_dev=False);

In [ ]:
az.compare({'asian_complete_pooling': ethnicity_complete_pooling_trace,
            'asian_no_pooling': ethnicity_no_pooling_trace,
            'asian_multilevel': ethnicity_multilevel_trace
            })

In [ ]:
az.plot_compare(az.compare({'asian_complete_pooling': ethnicity_complete_pooling_trace,
            'asian_no_pooling': ethnicity_no_pooling_trace,
            'asian_multilevel': ethnicity_multilevel_trace
            }), insample_dev=False);

In [ ]:
#pm.compare({'gender': trace_gender, 'white': trace_white}, ic='waic')

In [ ]:
# az.plot_compare(pm.compare({'gender': trace_gender, 'white': trace_white}, ic='waic'), insample_dev=False);

## 2C

In [ ]:
loss_gender_complete_pooling, loss_gender_nopooling, loss_gender_multilevel = cv_error(gender_features, df, )

## 3A

In [ ]:
posterior_samples_list_gender = gender_multilevel_trace.posterior["beta_Gender_Male_sigma"].values.tolist()
posterior_samples_gender = []
posterior_samples_gender.extend(posterior_samples_list_gender[0])
posterior_samples_gender.extend(posterior_samples_list_gender[1])

posterior_samples_list_asian = ethnicity_multilevel_trace.posterior["beta_Ethnicity_Asian_sigma"].values.tolist()
posterior_samples_asian = []
posterior_samples_asian.extend(posterior_samples_list_asian[0])
posterior_samples_asian.extend(posterior_samples_list_asian[1])


# Create the ridge plot
fig = ridgeplot(
    samples=[posterior_samples_gender, posterior_samples_asian],
    colorscale='Plotly3',
    colormode='mean-means',  # You can adjust this based on your preference
    linewidth=1.0,
    spacing=1.2,
    show_yticklabels=True,
    xpad=0.05,
    labels=['Gender', 'Asian']
)

fig.update_layout(
    height=500,
    width=800,
    font_size=16,
    plot_bgcolor="white",
    #xaxis_tickvals=[0, 1],
    #xaxis_ticktext=["0", "1"],
    xaxis_gridcolor="rgba(0, 0, 0, 0.1)",
    yaxis_gridcolor="rgba(0, 0, 0, 0.1)",
    #yaxis_title="Assigned Probability (%)",
    showlegend=False,
    title='Sigma of latent distributions'
)

# Show the figure
fig.show()


## 3B

In [ ]:
summary_gender = pm.summary(gender_no_pooling_trace)
gender_betas = summary_gender[summary_gender.index.str.startswith("beta_Gender_Male")]['mean']

In [ ]:
summary_asian = pm.summary(ethnicity_no_pooling_trace)
asian_betas = summary_asian[summary_asian.index.str.startswith("beta_Ethnicity_Asian")]['mean']

In [ ]:
def bootstrap(data):
    B = 10000

    bootstrapped_means = np.zeros(B)
    bootstrapped_std = np.zeros(B)

    # Perform bootstrapping
    for i in range(B):
        # Generate a bootstrap sample by sampling with replacement
        bootstrap_sample = np.random.choice(data, size=len(data), replace=True)

        # Calculate the mean of the bootstrap sample
        bootstrapped_means[i] = np.mean(bootstrap_sample)
        bootstrapped_std[i] = np.std(bootstrap_sample)

    return bootstrapped_means, bootstrapped_std

In [ ]:
bootstrap_means_gender_betas, bootstrap_std_gender_betas = bootstrap(gender_betas)
bootstrap_means_asian_betas, bootstrap_std_asian_betas =  bootstrap(asian_betas)

In [ ]:
plt.hist(bootstrap_std_gender_betas, bins=20)

In [ ]:
plt.hist(bootstrap_std_asian_betas, bins=20)